In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout
from keras.metrics import Recall,Precision

In [2]:
train_data_path = './MVSA/train'
val_data_path = './MVSA/val'
test_data_path = './MVSA/test'
weights_path = './weight/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [3]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_state()
        self.recall.reset_state()

f1_score = F1Score()

In [4]:
from keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'best_multi_model_f1_score.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',  # Save the model when the validation loss is minimized
    save_best_only=True,  # Save only the best model
    mode='max',  # Minimize the validation loss
    verbose=1,  # Print a message when the best model is saved
)

In [5]:
input_shape = (224, 224, 3)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        val_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 13702 images belonging to 3 classes.
Found 1713 images belonging to 3 classes.
Found 1714 images belonging to 3 classes.


In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

initial_learning_rate = 1e-4
optimizer = Adam(learning_rate=initial_learning_rate)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

In [10]:
base_model = VGG16(weights=None, include_top=False, input_shape=input_shape, classes=365)
base_model.load_weights(weights_path)
base_model.layers.pop()
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-2:]:
    layer.trainable = True

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_score])

In [21]:
# from keras.models import load_model

# custom_objects = {'F1Score': F1Score}
# model = load_model('best_multi_model_f1_score.h5', custom_objects=custom_objects)

In [11]:
# Train the model
model.fit_generator(
        train_generator,
        steps_per_epoch=12000 // 32,
        epochs=50,
        validation_data=val_generator,
        validation_steps=1600 // 32,
        callbacks=[checkpoint_callback,lr_scheduler])



C:\Users\Jeffan\AppData\Local\Temp\ipykernel_19808\1316729814.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
375/375 [==============================] - ETA: 0s - loss: 0.9438 - accuracy: 0.5240 - f1_score: 0.4572

c:\Users\Jeffan\.conda\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()



Epoch 1: val_accuracy improved from -inf to 0.60125, saving model to best_multi_model_f1_score.h5
375/375 [==============================] - 1332s 4s/step - loss: 0.9438 - accuracy: 0.5240 - f1_score: 0.4572 - val_loss: 0.8815 - val_accuracy: 0.6012 - val_f1_score: 0.3933 - lr: 1.0000e-04
Epoch 2/50
375/375 [==============================] - ETA: 0s - loss: 0.8717 - accuracy: 0.5559 - f1_score: 0.4965
Epoch 2: val_accuracy improved from 0.60125 to 0.60312, saving model to best_multi_model_f1_score.h5
375/375 [==============================] - 1333s 4s/step - loss: 0.8717 - accuracy: 0.5559 - f1_score: 0.4965 - val_loss: 0.8859 - val_accuracy: 0.6031 - val_f1_score: 0.3488 - lr: 1.0000e-04
Epoch 3/50
375/375 [==============================] - ETA: 0s - loss: 0.8602 - accuracy: 0.5742 - f1_score: 0.5217
Epoch 3: val_accuracy did not improve from 0.60312
375/375 [==============================] - 1328s 4s/step - loss: 0.8602 - accuracy: 0.5742 - f1_score: 0.5217 - val_loss: 0.8531 - val_

KeyboardInterrupt: 

In [12]:
test_loss, test_accuracy,test_f1_score = model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_accuracy)
print('Test f1_score:', test_f1_score)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_19808\2842752893.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy,test_f1_score = model.evaluate_generator(test_generator, steps=50)


Test accuracy: 0.581250011920929
Test f1_score: 0.499066025018692


In [13]:
from keras.models import load_model

custom_objects = {'F1Score': F1Score}
best_model = load_model('best_multi_model_f1_score.h5', custom_objects=custom_objects)
test_loss, test_accuracy, test_f1_score = best_model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_accuracy)
print('Test f1_score:', test_f1_score)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_19808\3555468199.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy, test_f1_score = best_model.evaluate_generator(test_generator, steps=50)


Test accuracy: 0.5849999785423279
Test f1_score: 0.4713275134563446


In [ ]:
from sklearn.metrics import f1_score

predict_result = np.argmax(model.predict_generator(test_generator), axis=-1)
actual_result = test_generator.classes

f1_score(actual_result, predict_result, average='micro')

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_12980\311859083.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict_result = np.argmax(model.predict_generator(test_generator), axis=-1)


0.5221703617269545

In [ ]:
f1_score(actual_result, predict_result, average=None)

array([0.        , 0.        , 0.58785249])

In [ ]:
train_generator.class_indices

{'negative': 0, 'neutral': 1, 'positive': 2}

In [ ]:
import pandas as pd
pd.Series(train_generator.classes).value_counts()

2    7697
1    5177
0     828
dtype: int64